In [1]:
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 14.5 MB/s eta 0:00:00


In [2]:
import nibabel as nib
import SimpleITK as sitk
import glob
import os

In [4]:
input_folder = "path/to/input_folder"
output_folder = "path/to/output_folder"

In [6]:
def correct_image(nii_file_path, output_folder):
    nii_image = nib.load(nii_file_path)
    image_data = nii_image.get_fdata()
    sitk_img = sitk.GetImageFromArray(image_data)


    transformed = sitk.RescaleIntensity(sitk_img, 0, 255)
    transformed = sitk.LiThreshold(sitk_img, 0, 1)
    head_mask = transformed

    shrinkFactor = 4
    inputImage = sitk.Shrink(sitk_img, [shrinkFactor] * sitk_img.GetDimension())
    maskImage = sitk.Shrink(head_mask, [shrinkFactor] * sitk_img.GetDimension())

    bias_corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrected_img = bias_corrector.Execute(inputImage, maskImage)


    output_filename = os.path.join(output_folder, os.path.basename(nii_file_path).replace('.nii', '_after_n4_correction.nii'))


    sitk.WriteImage(corrected_img, output_filename)

In [5]:
for nii_file_path in glob.glob(input_folder + '/*.nii'):
    correct_image(nii_file_path, output_folder)